# 指定した放電番号のデータを切り出す

In [13]:
from os.path import join
import pandas as pd
import aklab.qulee as qulee
import numpy as np

from quest_pp_calibration.convert_qms_time import t2sa

In [14]:
shot_start = 34313
shot_end = 34320
date = 170112

pp_path = f'../../data/raw/pdp8_shot_txt'
csv_path = "../../data/raw/pdp8_csv/S1_170112_091831.CSV"

### 取得データの時間計算

指定したショット番号からデータを抽出する時間を取得

In [15]:
# 抽出する時間を計算
def arrange_csv(shot, date, csv_path):
    filename1 = f'{shot} PDP 8.txt'
    filename2 = f'{shot+1} PDP 8.txt'

    df1 = pd.read_csv(join(pp_path,filename1),delimiter='\t')
    df2 = pd.read_csv(join(pp_path,filename2),delimiter='\t')

    start_time = df1[df1.columns[2]][0]

    if shot == 34320:
        end_time = '002:10:14.968'
    else:
        end_time = df2[df2.columns[2]][0]

    print(shot)

    qms = qulee.QMS(csv_path)
    qms.load_data()
    df_qms = qms.data

    start_time = t2sa([start_time])
    end_time = t2sa([end_time])

    x = np.array(df_qms['tsec'])
    y = np.array(df_qms['m2'])

    start_idx = np.abs(x - start_time).argmin()
    end_idx = np.abs(x - end_time).argmin()
    i1, i2 = sorted([start_idx, end_idx])

    # 範囲抽出
    x_new = x[i1:i2+1]
    y_new = y[i1:i2+1]

    time = x_new - x_new[0]

    # 元のデータと開始時間を合わせる
    start_time_origin = df1[df1.columns[0]][0]
    time += start_time_origin

    spth = f'../../data/extract_shot/{date}'
    fpth = join(spth, f'{shot}_PDP8_extracted.txt')

    np.savetxt(
        fpth,
        np.array([x_new, time, y_new]).T
    )

In [16]:
for i in list(range(shot_start, shot_end+1)):
    arrange_csv(i, date, csv_path)

34313
34314
34315
34316
34317
34318
34319
34320
